# Acquire

In [1]:
from wrangle_zillow import *
import pandas as pd
pd.options.display.max_rows = 65 

In [2]:
zillow = get_zillow()

In [3]:
encodings, data_dict = get_dict()

In [4]:
df_summary(zillow)

--- Shape: (77381, 61)
--- Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77381 entries, 0 to 77380
Data columns (total 61 columns):
logerror                        77381 non-null float64
transactiondate                 77381 non-null object
id                              77381 non-null int64
parcelid                        77381 non-null int64
airconditioningtypeid           24953 non-null float64
architecturalstyletypeid        206 non-null float64
basementsqft                    50 non-null float64
bathroomcnt                     77381 non-null float64
bedroomcnt                      77381 non-null float64
buildingclasstypeid             15 non-null float64
buildingqualitytypeid           49672 non-null float64
calculatedbathnbr               76772 non-null float64
decktypeid                      614 non-null float64
finishedfloor1squarefeet        6023 non-null float64
calculatedfinishedsquarefeet    77185 non-null float64
finishedsquarefeet12            73749 non-null flo

   num_cols_missing    pct_cols_missing  num_rows
0                19  31.147540983606557         2
1                20   32.78688524590164        10
2                21   34.42622950819672        83
3                22    36.0655737704918       262
4                23  37.704918032786885       599
5                24   39.34426229508197       660
6                25   40.98360655737705      5768
7                26   42.62295081967213      7996
8                27   44.26229508196721     12410
9                28   45.90163934426229     20660
10               29  47.540983606557376     16098
11               30   49.18032786885246      6391
12               31   50.81967213114754      3604
13               32  52.459016393442624      2089
14               33   54.09836065573771       300
15               34   55.73770491803278       230
16               35  57.377049180327866        29
17               36   59.01639344262295        23
18               37   60.65573770491803        28


hashottuborspa:
1.0    1538
Name: hashottuborspa, dtype: int64
heatingorsystemtypeid:
2.0     33550
7.0     14065
24.0      846
6.0       776
20.0      104
13.0       59
18.0       23
1.0        13
10.0        3
11.0        1
Name: heatingorsystemtypeid, dtype: int64
latitude:
(33338047.759000003, 33487451.0]     1045
(33487451.0, 33635375.0]             5318
(33635375.0, 33783299.0]            10047
(33783299.0, 33931223.0]            14789
(33931223.0, 34079147.0]            14222
(34079147.0, 34227071.0]            18893
(34227071.0, 34374995.0]             6261
(34374995.0, 34522919.0]             3146
(34522919.0, 34670843.0]             2321
(34670843.0, 34818767.0]             1339
Name: latitude, dtype: int64
longitude:
(-119477336.781, -119283338.0]      221
(-119283338.0, -119091260.0]       1997
(-119091260.0, -118899182.0]       1494
(-118899182.0, -118707104.0]       2660
(-118707104.0, -118515026.0]       5926
(-118515026.0, -118322948.0]      15424
(-118322948.0, -118130

Notes:
- many columns have loads of nulls. Probably going to drop those
- transactiondate is an object instead of some sort of number or date, so it can't sort or display max/min
- There appears to be a 2018 transaction date
- id is totally uninformative
- nulls for basementsqft probably mean no basement
- some of the nulls for AC type id might mean no AC, but there's no way over half the listings lack AC
- someone has a 3200+ sqft basement, 2800+ sqft basement is pretty huge too
- only 12 people with 9+ bathrooms
- 18 with 12+ bedrooms
- buildingclasstypeid doesn't seem particularly useful
- buildingqualitytypeid is really a scale (apparently from 1 to 12??) of how well-built the building is
- decktypeid seems super useless. 614 have type 16 deck which isn't defined anywhere.
- There are several different sqft metrics. calculated sames most useful based on the definitions in data_dict
- some buildings seem to be way too large to be single-unit. There are 23 above 10.7k sqft and 1 over 30k
- there are some really high garagecarcnts
- there are a fair few really large garages
- nulls in hashottuborspa appear to mean that they don't have hot tubs or spas
- holy cannoli there are some massive lot sizes. the 7 with >6M sqft lot size appear to be mislabeled. Perhaps investigate the other high ones later. The 7 houses referenced are pictured below. I believe the entire subdivision lot was listed as the lot size.
- pooltype columns refer to whether the pool is just pool, just hot tub, or both. it's like onehotencoder of pooltype kind of.
- propertylandusecode is county zoning stuff. could be useful in identifying single-unit properties. Probably differs across counties? worth investigating
- If no unit count, drop these propertylandusedescs: residential general, duplex, quadruplex, triplex, cluster home, commerical/office/residential mixed use
- propertyzoningdesc is another zoning column? not sure on the differences between these three
- rawcensustractandblock and regionidcity should probably be categorical, and others involved in this fips categorization
- Very unclear what 0 roomcnt means since they appear to have nonzero bathroomcnt and bedroomcnt. 
- storytypeid relays no information because it only indicates whether a building has a basement, which is already indicated by a non-null value in basementsqft
- really not seeing the value in typeconstructiontypeid. similarly unclear what a null value here indicates
- of course we will drop all unitcnts > 1, and make an educated guess about which null unitcnt properties really have a unitcnt of 1 based on the value_counts within the unitcnt==1 cohort
- there are some strangely large sheds based on yardbuildingsqft. I'm thinking the particularly large ones probably indicate unitcnt > 1 but we shall see
- yearbuilt could influence value. not really going to influence what I drop/don't.
- Six-story house is somehow single-family residential, zoning is unclear but based on comparison of unitcnt == 1 and unitcnt > 1 cohorts, it appears to actually be single-unit. very strange. Latitude, longitude search on google maps shows an elementary school. There is no building in the vicinity that has >1 story.
- fireplaceflag indicates whether or not there is a fireplace at the houses in which fireplacecnt is null.
- probably drop the particularly expensive properties
- null taxdelinquencyflag means their taxes are not past due as of 2015
- taxdelinquencyyear is year from which your delinquent taxes are due. Should be restructured to be a four digit number instead of '99, '00, '01, etc.
- anything with null calculatedfinishedsquarefeet should be dropped.

<img src='huge_lots.png'>

In [5]:
for frame in encodings:
    print(encodings[frame])

    airconditioningtypeid airconditioningdesc
0                       1             Central
1                       2       Chilled Water
2                       3  Evaporative Cooler
3                       4         Geo Thermal
4                       5                None
5                       6               Other
6                       7    Packaged AC Unit
7                       8             Partial
8                       9       Refrigeration
9                      10         Ventilation
10                     11           Wall Unit
11                     12         Window Unit
12                     13                 Yes
    architecturalstyletypeid         architecturalstyledesc
0                          1                        A-Frame
1                          2                       Bungalow
2                          3                       Cape Cod
3                          4                        Cottage
4                          5                       Colon

In [6]:
df_summary(zillow[zillow.unitcnt == 1])

--- Shape: (47293, 61)
--- Info
<class 'pandas.core.frame.DataFrame'>
Int64Index: 47293 entries, 3 to 77380
Data columns (total 61 columns):
logerror                        47293 non-null float64
transactiondate                 47293 non-null object
id                              47293 non-null int64
parcelid                        47293 non-null int64
airconditioningtypeid           22020 non-null float64
architecturalstyletypeid        0 non-null float64
basementsqft                    0 non-null float64
bathroomcnt                     47293 non-null float64
bedroomcnt                      47293 non-null float64
buildingclasstypeid             8 non-null float64
buildingqualitytypeid           46803 non-null float64
calculatedbathnbr               47247 non-null float64
decktypeid                      0 non-null float64
finishedfloor1squarefeet        0 non-null float64
calculatedfinishedsquarefeet    47286 non-null float64
finishedsquarefeet12            47246 non-null float64
fini

(130.222, 2329.7]     39578
(2329.7, 4507.4]       6932
(4507.4, 6685.1]        590
(6685.1, 8862.8]         94
(8862.8, 11040.5]        35
(11040.5, 13218.2]       11
(13218.2, 15395.9]        2
(15395.9, 17573.6]        2
(17573.6, 19751.3]        0
(19751.3, 21929.0]        2
Name: finishedsquarefeet12, dtype: int64
finishedsquarefeet13:
Series([], Name: finishedsquarefeet13, dtype: int64)
finishedsquarefeet15:
744.0      2
608.0      2
2044.0     1
2168.0     1
600.0      1
598.0      1
1576.0     1
2878.0     1
1566.0     1
654.0      1
1857.0     1
1042.0     1
1040.0     1
646.0      1
1160.0     1
2010.0     1
1344.0     1
2596.0     1
3204.0     1
893.0      1
1224.0     1
1226.0     1
1000.0     1
1101.0     1
2460.0     1
1748.0     1
2472.0     1
35640.0    1
1242.0     1
1121.0     1
3272.0     1
826.0      1
1182.0     1
Name: finishedsquarefeet15, dtype: int64
finishedsquarefeet50:
Series([], Name: finishedsquarefeet50, dtype: int64)
finishedsquarefeet6:
1380.0    1
1482

(59948352007016.984, 102636920499301.5]    47176
(102636920499301.5, 144902829897603.0]         0
(144902829897603.0, 187168739295904.5]         0
(187168739295904.5, 229434648694206.03]        0
(229434648694206.03, 271700558092507.5]        0
(271700558092507.5, 313966467490809.0]         0
(313966467490809.0, 356232376889110.5]         0
(356232376889110.5, 398498286287411.94]        0
(398498286287411.94, 440764195685713.5]        0
(440764195685713.5, 483030105084015.0]         1
Name: censustractandblock, dtype: int64
None


In [7]:
df_summary(zillow[zillow.unitcnt > 1])

--- Shape: (3270, 61)
--- Info
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3270 entries, 11 to 77342
Data columns (total 61 columns):
logerror                        3270 non-null float64
transactiondate                 3270 non-null object
id                              3270 non-null int64
parcelid                        3270 non-null int64
airconditioningtypeid           219 non-null float64
architecturalstyletypeid        0 non-null float64
basementsqft                    0 non-null float64
bathroomcnt                     3270 non-null float64
bedroomcnt                      3270 non-null float64
buildingclasstypeid             4 non-null float64
buildingqualitytypeid           2735 non-null float64
calculatedbathnbr               2958 non-null float64
decktypeid                      1 non-null float64
finishedfloor1squarefeet        5 non-null float64
calculatedfinishedsquarefeet    3270 non-null float64
finishedsquarefeet12            98 non-null float64
finishedsquarefeet1

(365.72900000000004, 2211.0]    1749
(2211.0, 4038.0]                1269
(4038.0, 5865.0]                 236
(5865.0, 7692.0]                  11
(7692.0, 9519.0]                   2
(9519.0, 11346.0]                  1
(11346.0, 13173.0]                 0
(13173.0, 15000.0]                 0
(15000.0, 16827.0]                 0
(16827.0, 18654.0]                 2
Name: calculatedfinishedsquarefeet, dtype: int64
finishedsquarefeet12:
(906.269, 2697.0]     67
(2697.0, 4470.0]      26
(4470.0, 6243.0]       3
(6243.0, 8016.0]       0
(8016.0, 9789.0]       1
(9789.0, 11562.0]      0
(11562.0, 13335.0]     0
(13335.0, 15108.0]     0
(15108.0, 16881.0]     0
(16881.0, 18654.0]     1
Name: finishedsquarefeet12, dtype: int64
finishedsquarefeet13:
Series([], Name: finishedsquarefeet13, dtype: int64)
finishedsquarefeet15:
(602.422, 2377.7]     1805
(2377.7, 4135.4]       965
(4135.4, 5893.1]       174
(5893.1, 7650.8]         9
(7650.8, 9408.5]         1
(9408.5, 11166.2]        1
(11166.2,

(60370272026103.0, 60444918591306.3]     2972
(60444918591306.3, 60518826081606.6]        0
(60518826081606.6, 60592733571906.9]      220
(60592733571906.9, 60666641062207.2]        0
(60666641062207.2, 60740548552507.51]       0
(60740548552507.51, 60814456042807.8]       0
(60814456042807.8, 60888363533108.1]        0
(60888363533108.1, 60962271023408.4]        0
(60962271023408.4, 61036178513708.7]        0
(61036178513708.7, 61110086004009.0]       73
Name: censustractandblock, dtype: int64
None


Notes to differentiate unitcnt = 1 and other unitcnts
- Decision Tree to differentiate?
- 